In [156]:
import pandas as pd
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import EfficientNetB0


In [148]:
image_height = 64
image_width = 64
batch_size = 20
num_epochs = 10

In [60]:
# Load the data
df = pd.read_csv('/Users/Timothe/Documents/DSB/QB/data/train_data/metadata.csv')
df["path"] = "train_data/" + df["path"] + ".tif"
df.head()

,date,id_coord,plume,set,lat,lon,coord_x,coord_y,path
0,20230223,id_6675,yes,train,31.528750,74.330625,24,47,train_data/images/plume/20230223_methane_mixin...
1,20230103,id_2542,yes,train,35.538000,112.524000,42,37,train_data/images/plume/20230103_methane_mixin...
2,20230301,id_6546,yes,train,21.060000,84.936667,58,15,train_data/images/plume/20230301_methane_mixin...
3,20230225,id_6084,yes,train,26.756667,80.973333,28,62,train_data/images/plume/20230225_methane_mixin...
4,20230105,id_2012,yes,train,34.800000,40.770000,59,44,train_data/images/plume/20230105_methane_mixin...


In [125]:
# Split the data
train_df, valid_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(valid_df, test_size=0.5, random_state=42)

In [126]:
# Load and preprocess images
train_images = []
train_labels = []

for index, row in train_df.iterrows():
    image = Image.open(row['path'])
    image = image.resize((image_height, image_width))
    image = np.array(image) / 255.0  # Rescale pixel values to [0, 1]
    train_images.append(image)
    train_labels.append(row['plume'])

train_images = np.array(train_images)
train_labels = np.array(train_labels)

validation_images = [] 
validation_labels = []

for index, row in val_df.iterrows():
    image = Image.open(row['path'])
    image = image.resize((image_height, image_width))
    image = np.array(image) / 255.0  # Rescale pixel values to [0, 1]
    validation_images.append(image)
    validation_labels.append(row['plume'])

validation_images = np.array(validation_images)
validation_labels = np.array(validation_labels)

test_images = []  # Load and preprocess test images similarly
test_labels = []

for index, row in test_df.iterrows():
    image = Image.open(row['path'])
    image = image.resize((image_height, image_width))
    image = np.array(image) / 255.0  # Rescale pixel values to [0, 1]
    test_images.append(image)
    test_labels.append(row['plume'])

test_images = np.array(test_images)
test_labels = np.array(test_labels)


In [127]:
# Convert string labels to numerical format in your data frames
train_df['plume'] = train_df['plume'].map({'yes': 1, 'no': 0})
val_df['plume'] = val_df['plume'].map({'yes': 1, 'no': 0})
test_df['plume'] = test_df['plume'].map({'yes': 1, 'no': 0})

# Now convert the labels to NumPy arrays
train_labels = np.array(train_df['plume'])
validation_labels = np.array(val_df['plume'])
test_labels = np.array(test_df['plume'])

In [128]:
# Define the CNN model
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 1)))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

# Flatten the feature maps
model.add(Flatten())

# Dense layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])

# Train the model
epochs = 15  # Adjust the number of training epochs as needed
model.fit(train_images, train_labels, validation_data=(validation_images, validation_labels), epochs=epochs)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test accuracy:", test_accuracy)


Epoch 1/15
10/10 [==============================] - 2s 103ms/step - loss: 8.9623 - auc: 0.4972 - val_loss: 0.7326 - val_auc: 0.8651
Epoch 2/15
10/10 [==============================] - 1s 67ms/step - loss: 0.8028 - auc: 0.5986 - val_loss: 0.6146 - val_auc: 0.8272
Epoch 3/15
10/10 [==============================] - 1s 68ms/step - loss: 0.6147 - auc: 0.7156 - val_loss: 0.6499 - val_auc: 0.8651
Epoch 4/15
10/10 [==============================] - 1s 68ms/step - loss: 0.5083 - auc: 0.8655 - val_loss: 0.5286 - val_auc: 0.7790
Epoch 5/15
10/10 [==============================] - 1s 67ms/step - loss: 0.4379 - auc: 0.8957 - val_loss: 0.4900 - val_auc: 0.8221
Epoch 6/15
10/10 [==============================] - 1s 68ms/step - loss: 0.3368 - auc: 0.9310 - val_loss: 0.4876 - val_auc: 0.8703
Epoch 7/15
10/10 [==============================] - 1s 68ms/step - loss: 0.2545 - auc: 0.9664 - val_loss: 0.4693 - val_auc: 0.8754
Epoch 8/15
10/10 [==============================] - 1s 66ms/step - loss: 0.2019 - 

In [129]:
# Create an instance of the ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255  # Rescale pixel values
)

In [130]:
train_images = train_images.reshape(train_images.shape + (1,))
train_datagen = datagen.flow(train_images, train_labels, batch_size=batch_size)
validation_images = validation_images.reshape(validation_images.shape + (1,))
validation_datagen = datagen.flow(validation_images, validation_labels, batch_size=batch_size)
test_images = test_images.reshape(test_images.shape + (1,))
test_datagen = datagen.flow(test_images, test_labels, batch_size=batch_size)

In [168]:
# Define the CNN model
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 1)))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

# Flatten the feature maps
model.add(Flatten())

# Dense layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])

# Train the model
epochs = 15  # Adjust the number of training epochs as needed
model.fit(train_datagen, validation_data=validation_datagen, epochs=epochs)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_datagen)
print("Test accuracy:", test_accuracy)

Epoch 1/15
10/10 [==============================] - 4s 138ms/step - loss: 0.6964 - auc: 0.5789 - val_loss: 0.6901 - val_auc: 0.8410
Epoch 2/15
10/10 [==============================] - 1s 88ms/step - loss: 0.6745 - auc: 0.6624 - val_loss: 0.7232 - val_auc: 0.8723
Epoch 3/15
10/10 [==============================] - 1s 86ms/step - loss: 0.6595 - auc: 0.7512 - val_loss: 0.6702 - val_auc: 0.8949
Epoch 4/15
10/10 [==============================] - 1s 79ms/step - loss: 0.6309 - auc: 0.8104 - val_loss: 0.6423 - val_auc: 0.8703
Epoch 5/15
10/10 [==============================] - 1s 78ms/step - loss: 0.6160 - auc: 0.8048 - val_loss: 0.6080 - val_auc: 0.8923
Epoch 6/15
10/10 [==============================] - 1s 78ms/step - loss: 0.6344 - auc: 0.7223 - val_loss: 0.6731 - val_auc: 0.9287
Epoch 7/15
10/10 [==============================] - 1s 80ms/step - loss: 0.6175 - auc: 0.7542 - val_loss: 0.5511 - val_auc: 0.9031
Epoch 8/15
10/10 [==============================] - 1s 75ms/step - loss: 0.5513 - 

In [137]:
# Split the data
train_df, valid_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(valid_df, test_size=0.5, random_state=42)

In [138]:
# Load and preprocess images
train_images = []
train_labels = []

for index, row in train_df.iterrows():
    image = Image.open(row['path']).convert('RGB')
    image = image.resize((image_height, image_width))
    image = np.array(image) / 255.0  # Rescale pixel values to [0, 1]
    train_images.append(image)
    train_labels.append(row['plume'])

train_images = np.array(train_images)
train_labels = np.array(train_labels)

validation_images = [] 
validation_labels = []

for index, row in val_df.iterrows():
    image = Image.open(row['path']).convert('RGB')
    image = image.resize((image_height, image_width))
    image = np.array(image) / 255.0  # Rescale pixel values to [0, 1]
    validation_images.append(image)
    validation_labels.append(row['plume'])

validation_images = np.array(validation_images)
validation_labels = np.array(validation_labels)

test_images = []  # Load and preprocess test images similarly
test_labels = []

for index, row in test_df.iterrows():
    image = Image.open(row['path']).convert('RGB')
    image = image.resize((image_height, image_width))
    image = np.array(image) / 255.0  # Rescale pixel values to [0, 1]
    test_images.append(image)
    test_labels.append(row['plume'])

test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [150]:
pretrained_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

model = Sequential()
model.add(pretrained_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Freeze the pre-trained layers for fine-tuning
for layer in pretrained_model.layers:
    layer.trainable = False

In [152]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])

history = model.fit(train_images, 
                    train_labels,  
                    validation_data=(validation_images, validation_labels), 
                    epochs=num_epochs)

test_loss, test_accuracy = model.evaluate(test_images, test_labels)


Epoch 1/10
10/10 [==============================] - 8s 492ms/step - loss: 0.6666 - auc: 0.6491 - val_loss: 0.5878 - val_auc: 0.8149
Epoch 2/10
10/10 [==============================] - 4s 392ms/step - loss: 0.5026 - auc: 0.8408 - val_loss: 0.5112 - val_auc: 0.8615
Epoch 3/10
10/10 [==============================] - 5s 497ms/step - loss: 0.4290 - auc: 0.8982 - val_loss: 0.5442 - val_auc: 0.9005
Epoch 4/10
10/10 [==============================] - 5s 483ms/step - loss: 0.3905 - auc: 0.9201 - val_loss: 0.6095 - val_auc: 0.9087
Epoch 5/10
10/10 [==============================] - 5s 461ms/step - loss: 0.3559 - auc: 0.9343 - val_loss: 0.4722 - val_auc: 0.9087
Epoch 6/10
10/10 [==============================] - 5s 463ms/step - loss: 0.2947 - auc: 0.9689 - val_loss: 0.4437 - val_auc: 0.9097
Epoch 7/10
10/10 [==============================] - 5s 469ms/step - loss: 0.2618 - auc: 0.9786 - val_loss: 0.3999 - val_auc: 0.9087
Epoch 8/10
10/10 [==============================] - 5s 481ms/step - loss: 0.

In [169]:
# Load the pre-trained EfficientNetB0 model
pretrained_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

model = Sequential()
model.add(pretrained_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Freeze the pre-trained layers for fine-tuning
for layer in pretrained_model.layers:
    layer.trainable = False

# Freeze the pre-trained layers for fine-tuning
for layer in pretrained_model2.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])

history = model.fit(train_images, 
                    train_labels,
                    validation_data=(validation_images, validation_labels), 
                    epochs=num_epochs)

test_loss, test_accuracy = model.evaluate(test_images, test_labels)

Epoch 1/10
10/10 [==============================] - 6s 444ms/step - loss: 0.6749 - auc: 0.6394 - val_loss: 0.5057 - val_auc: 0.8523
Epoch 2/10
10/10 [==============================] - 4s 429ms/step - loss: 0.5163 - auc: 0.8188 - val_loss: 0.4485 - val_auc: 0.8779
Epoch 3/10
10/10 [==============================] - 4s 450ms/step - loss: 0.4332 - auc: 0.8851 - val_loss: 0.4281 - val_auc: 0.8933
Epoch 4/10
10/10 [==============================] - 5s 508ms/step - loss: 0.3680 - auc: 0.9399 - val_loss: 0.4976 - val_auc: 0.9159
Epoch 5/10
10/10 [==============================] - 6s 653ms/step - loss: 0.3316 - auc: 0.9508 - val_loss: 0.4982 - val_auc: 0.9164
Epoch 6/10
10/10 [==============================] - 4s 408ms/step - loss: 0.2892 - auc: 0.9698 - val_loss: 0.4808 - val_auc: 0.9164
Epoch 7/10
10/10 [==============================] - 4s 429ms/step - loss: 0.2467 - auc: 0.9816 - val_loss: 0.4673 - val_auc: 0.9226
Epoch 8/10
10/10 [==============================] - 5s 498ms/step - loss: 0.

In [170]:
# Load the pre-trained EfficientNetB0 model
pretrained_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

model = Sequential()
model.add(pretrained_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Freeze the pre-trained layers for fine-tuning
for layer in pretrained_model.layers:
    layer.trainable = False

# Freeze the pre-trained layers for fine-tuning
for layer in pretrained_model2.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])

history = model.fit(train_datagen,
                    validation_data=validation_datagen, 
                    epochs=num_epochs)

test_loss, test_accuracy = model.evaluate(test_datagen)

Epoch 1/10


2023-10-11 17:26:31.024014: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at conv_ops_fused_impl.h:761 : INVALID_ARGUMENT: input depth must be evenly divisible by filter depth: 1 vs 3


InvalidArgumentError: Graph execution error:

Detected at node sequential_35/vgg16/block1_conv1/Relu defined at (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code

  File "/Users/Timothe/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/Timothe/Library/Python/3.9/lib/python/site-packages/traitlets/config/application.py", line 982, in launch_instance

  File "/Users/Timothe/Library/Python/3.9/lib/python/site-packages/ipykernel/kernelapp.py", line 712, in start

  File "/Users/Timothe/Library/Python/3.9/lib/python/site-packages/tornado/platform/asyncio.py", line 215, in start

  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/l